In [1]:
import datetime
import math
import os
import time
import pandas as pd
from actransit import ACTransit
BASE_DIR = os.path.abspath('')


ModuleNotFoundError: No module named 'actransit'

In [24]:
"""
A function to convert standard latitude
and longitude coordinates to a mercator
coordinate system
"""
def mercator(lat, lon):
    r_major = 6378137.000
    merc_lon = r_major * math.radians(lon)
    try:
        scale = merc_lon/lon
    except ZeroDivisionError:
        return None
    merc_lat = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + lat * (math.pi/180.0)/2.0)) * scale

    return (merc_lat, merc_lon)


In [25]:
map_dimension = {
    'berkeley_oakland': {
        'lat_range': (-13618281.04, -13600024.64),
        'lon_range': (4543915.73, 4565210.03)
    },
    'east_bay': {
        'lat_range': (-13629401.86, -13565426.54),
        'lon_range': (4500405.56, 4580273.45)
    }
}


In [26]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.annotations import Title
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import export_png

"""
A function to plot mercator coordinates over
SF East Bay map. Alter point alpha and color
to your desire.
"""
def plot_bokeh(x_list, y_list, count):
    # Set map boundaries (mercator) and axis type
    fig_map = map_dimension['east_bay']
    p = figure(x_range=fig_map['lat_range'],
               y_range=fig_map['lon_range'],
               x_axis_type="mercator", y_axis_type="mercator")
    
    # Tile Source for CartoDB Tile Service
    p.add_tile(CARTODBPOSITRON)
    # Set styling for Metro bus coordinate   
    p.square(x=y_list[1],
             y=x_list[1],
             color='blue',
             fill_alpha=0.01)
    
    p.circle(x=y_list[0],
             y=x_list[0],
             color='red',
             size=10,
             fill_alpha=.5)

    
    # Set chart title - date + time
    title = Title()
    title.text = f"AC Transit Buses - {datetime.datetime.now().strftime('%m/%d/%Y  %H:%M')}"
    p.title = title
    
    # For displaying map in notebook - comment out to avoid cluttering
    output_notebook()
    show(p)
    
    # Export as .png
    export_png(p, filename="plot{}.png".format(count))
    

In [27]:
"""
Find lat and lon of every bus.
Pass in an instance of AC Transit
GTFS real-time buses.
"""
def find_geotag(all_buses):
    lat_list = []
    lon_list = []
    # Gather lat, lon of every bus and convert
    # to mercator coord
    for bus in all_buses['entity']:
        vehicle_position = bus['vehicle']['position']
        lat = vehicle_position['latitude']
        lon = vehicle_position['longitude']
        tot_list = mercator(lat, lon)
        if not tot_list:
            continue
        lat_list.append(tot_list[0])
        lon_list.append(tot_list[1])
        
    return lat_list, lon_list


def find_stops(all_stops):
    lat_list = []
    lon_list = []
    # Gather lat, lon of every bus and convert
    # to mercator coord
    for stop in all_stops:
        lat = stop['Latitude']
        lon = stop['Longitude']
        tot_list = mercator(lat, lon)
        if not tot_list:
            continue
        lat_list.append(tot_list[0])
        lon_list.append(tot_list[1])
        
    return lat_list, lon_list


def time_sleep(t0, t1, seconds=10):
    # Induce {second} wait time
    time_difference = seconds - (t1 - t0)
    if time_difference <= 0:
        time.sleep(0)
    else:
        time.sleep(time_difference)


In [28]:
# Make client instance
client = ACTransit()

"""
Continually export bus coordinate images
every {seconds} to ./map_figs
"""
def main():
    os.chdir("{}/map_figs".format(BASE_DIR))
    img_count = 0

    all_bus = client.gtfsrt.vehicles()
    all_stops = client.stops.all()

    lat_bus, lon_bus = find_geotag(all_bus)
    lat_stop, lon_stop = find_stops(all_stops)
    lat = [lat_bus, lat_stop]
    lon = [lon_bus, lon_stop]
    print(lat,lon)
    plot_bokeh(lat, lon, img_count)

    os.chdir(BASE_DIR)


In [29]:
main()

Loading BokehJS ...

In [30]:
from actransit import ACTransit
ac = ACTransit()
len(ac.stops.all())

5359